In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder    # 导入LabelEncoder 工具 
from sklearn.preprocessing import OrdinalEncoder

In [3]:
names_gender_excel = pd.read_csv("nameGender.Shifenzheng.20050144.csv", on_bad_lines='skip')

In [6]:
chars = set("".join(names_gender_excel["Name"].astype(str).to_list()))

In [3]:
names_gender_excel

,Name,Gender
0,陈萌,M
1,贾新格,M
2,李锦花,F
3,张俊莲,F
4,曹守月,F
...,...,...
20050138,北京迪克森,NaN
20050139,安博北京国际贸易,NaN
20050140,广东腾骏动物,NaN
20050141,安徽福银金融服务,NaN


In [4]:
names_gender_excel.Gender.value_counts(dropna=False)

Gender
M           12773969
F            6479097
NaN           462411
              329153
N               4362
0               1020
1                119
#M                 9
19790522           1
#0449              1
                  1
Name: count, dtype: int64

In [5]:
# 过滤掉长度大于等于4的，因为这种可能是"厦门华瀚"这种公司名
# 过滤掉Gender不等与F或M的
filtered = names_gender_excel[names_gender_excel['Gender'].isin(['F','M']) ]

# 如果姓名长度为2，则填充一个空格到名字末尾
def padding(name):
    if len(str(name)) == 2:
        return name + "_"
    return name

filtered["Name"].apply(lambda x:  x + "_" if len(str(x)) == 2 else str(x) )

# 把姓名拆成3个字的vectors，如果不足三个，则填充'_'(上一步已经做好)
def nameSplit(name, index):
    arr = str(name)
    if arr and len(arr) > index:
        return arr[index]
    return ""
filtered["FamilyName"]        =filtered["Name"].map(lambda x: nameSplit(x, 0))
filtered["FirstNameChar"]     =filtered["Name"].map(lambda x: nameSplit(x, 1))
filtered["SecondNameChar"]    =filtered["Name"].map(lambda x: nameSplit(x, 2))


/var/folders/fd/120hdy_x5jgg_9v0_rv5k0l40000gn/T/ipykernel_25145/4066877819.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered["FamilyName"]        =filtered["Name"].map(lambda x: nameSplit(x, 0))
/var/folders/fd/120hdy_x5jgg_9v0_rv5k0l40000gn/T/ipykernel_25145/4066877819.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered["FirstNameChar"]     =filtered["Name"].map(lambda x: nameSplit(x, 1))
/var/folders/fd/120hdy_x5jgg_9v0_rv5k0l40000gn/T/ipykernel_25145/4066877819.py:21: SettingWithCo

In [7]:
filtered

,Name,Gender,FamilyName,FirstNameChar,SecondNameChar
0,陈萌,M,陈,萌,
1,贾新格,M,贾,新,格
2,李锦花,F,李,锦,花
3,张俊莲,F,张,俊,莲
4,曹守月,F,曹,守,月
...,...,...,...,...,...
20050115,姚兵,M,姚,兵,
20050116,王紫嫣,F,王,紫,嫣
20050117,杨易,F,杨,易,
20050118,Levenkova Anna,F,L,e,v


In [6]:
X_train = filtered[["FamilyName", "FirstNameChar", "SecondNameChar"]]
y_train = filtered[ "Gender"]

In [7]:
# all chars
all_chars = pd.concat([X_train["FamilyName"], X_train["FirstNameChar"], X_train["SecondNameChar"]])

# OrdinalEncoder
orde = OrdinalEncoder(dtype=np.int32, handle_unknown="use_encoded_value", unknown_value=-1)
orde.fit(all_chars.values.reshape(-1, 1))

OrdinalEncoder(dtype=<class 'numpy.int32'>, handle_unknown='use_encoded_value',
               unknown_value=-1)

In [8]:
X_train["FamilyName"]=orde.transform(X_train["FamilyName"].values.reshape(-1, 1))
X_train["FirstNameChar"]=orde.transform(X_train["FirstNameChar"].values.reshape(-1, 1))
X_train["SecondNameChar"]=orde.transform(X_train["SecondNameChar"].values.reshape(-1, 1))

/var/folders/fd/120hdy_x5jgg_9v0_rv5k0l40000gn/T/ipykernel_25145/362188169.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train["FamilyName"]=orde.transform(X_train["FamilyName"].values.reshape(-1, 1))
/var/folders/fd/120hdy_x5jgg_9v0_rv5k0l40000gn/T/ipykernel_25145/362188169.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train["FirstNameChar"]=orde.transform(X_train["FirstNameChar"].values.reshape(-1, 1))
/var/folders/fd/120hdy_x5jgg_9v0_rv5k0l40000gn/T/ipykernel_25145/362188169.py:3: Setting

# 算法1. 用LinearSVC来预测
用 sklearn.svm.LinearSVC做分类预测. 对比下来, SVC的准确度最高.

In [33]:
from sklearn.svm import LinearSVC

svc = LinearSVC()
svc.fit(X_train.iloc[:50000, :], y_train.iloc[:50000])

/Users/andrewy/anaconda3/envs/little_ai_zoo/lib/python3.11/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/Users/andrewy/anaconda3/envs/little_ai_zoo/lib/python3.11/site-packages/sklearn/svm/_base.py:1242: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


LinearSVC()

In [36]:
svc1m = LinearSVC()
svc1m.fit(X_train.iloc[:1000_0000, :], y_train.iloc[:1000_0000])

/Users/andrewy/anaconda3/envs/little_ai_zoo/lib/python3.11/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/Users/andrewy/anaconda3/envs/little_ai_zoo/lib/python3.11/site-packages/sklearn/svm/_base.py:1242: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


LinearSVC()

In [38]:
# pickle LinearSVC, dump失败
import pickle

# save
with open("svc_10m.pkl", "wb") as f:
    pickle.dump(svc1m, f)

# load
#with open("svc_50k.pkl", "rb") as f:
#    svc2 = pickle.load(f)

In [ ]:
import joblib

# save
joblib.dump(svc1m, "jl_svc_10m.pkl") 

# load
clf2 = joblib.load("jl_svc_10m.pkl")

In [26]:
y_train.iloc[:1000_0000].value_counts()

Gender
M    6693241
F    3306759
Name: count, dtype: int64

In [32]:
y_train.iloc[:100_0000].value_counts()

Gender
M    822672
F    177328
Name: count, dtype: int64

# 测试结果
很奇怪, M的precision为1.0, F的为0. 不理解(后面的分析发现测试数据发布有点问题, M远多于F, {M:9290,F:710})。

In [43]:
# 误差, https://blog.csdn.net/wangzi11111111/article/details/88614720
from sklearn.metrics import mean_absolute_error, accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import classification_report

predictions = svc.predict(X_train.iloc[50000: 510000, :])
y_true = y_train.iloc[50000: 510000]
acc = accuracy_score(predictions, y_true)
print("Accuracy score: {0}".format(acc))

cls_report = classification_report(predictions, y_true)
print("Classification report: \n{0}".format(cls_report))

Accuracy score: 0.849095652173913
Classification report: 
              precision    recall  f1-score   support

           F       0.05      0.10      0.06     24138
           M       0.95      0.89      0.92    435862

    accuracy                           0.85    460000
   macro avg       0.50      0.50      0.49    460000
weighted avg       0.90      0.85      0.87    460000



# 算法2. 用RandomForest来做分类
用RandomForest来做分类.

In [11]:
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier(n_estimators=5000)
rfc.fit(X_train.iloc[:50000, :], y_train.iloc[:50000])

RandomForestClassifier(n_estimators=5000)

In [13]:
# 误差, https://blog.csdn.net/wangzi11111111/article/details/88614720
from sklearn.metrics import mean_absolute_error, accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import classification_report

predictions = rfc.predict(X_train.iloc[500000: 510000, :])
y_true = y_train.iloc[500000: 510000]
acc = accuracy_score(predictions, y_true)
print("Accuracy score: {0}".format(acc))

cls_report = classification_report(predictions, y_true)
print("Classification report: \n{0}".format(cls_report))

Accuracy score: 0.7917
Classification report: 
              precision    recall  f1-score   support

           F       0.49      0.17      0.25      2067
           M       0.81      0.95      0.88      7933

    accuracy                           0.79     10000
   macro avg       0.65      0.56      0.56     10000
weighted avg       0.75      0.79      0.75     10000



# 算法3. 用GradientBoostingClassifier做分类

In [15]:
from sklearn.ensemble import GradientBoostingClassifier

gbc = GradientBoostingClassifier(n_estimators=2000)

gbc.fit(X_train.iloc[:50000, :], y_train.iloc[:50000])

GradientBoostingClassifier(n_estimators=2000)

In [16]:
# 误差, https://blog.csdn.net/wangzi11111111/article/details/88614720
from sklearn.metrics import mean_absolute_error, accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import classification_report

predictions = gbc.predict(X_train.iloc[500000: 510000, :])
y_true = y_train.iloc[500000: 510000]
acc = accuracy_score(predictions, y_true)
print("Accuracy score: {0}".format(acc))

cls_report = classification_report(predictions, y_true)
print("Classification report: \n{0}".format(cls_report))

Accuracy score: 0.7967
Classification report: 
              precision    recall  f1-score   support

           F       0.53      0.18      0.27      2077
           M       0.82      0.96      0.88      7923

    accuracy                           0.80     10000
   macro avg       0.67      0.57      0.58     10000
weighted avg       0.76      0.80      0.75     10000



In [21]:
y_true.value_counts(dropna=False)

Gender
M    9290
F     710
Name: count, dtype: int64

# 算法3. 用Tensorflow做分类预测
接下来用tensorflow来实现性别预测